In [1]:
import numpy as np
import pandas as pd
import itertools
import sys

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.optimizers.schedules import ExponentialDecay

from sklearn.metrics import mean_absolute_error as mae
from sklearn.preprocessing import RobustScaler, normalize
from sklearn.model_selection import train_test_split, GroupKFold, KFold

from IPython.display import display

import warnings

warnings.simplefilter('ignore')
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [2]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
if 'google.colab' in sys.modules:
  
  # 環境準備
  from googleapiclient.discovery import build
  import io, os
  from googleapiclient.http import MediaIoBaseDownload
  from google.colab import auth

  auth.authenticate_user()

  drive_service = build('drive', 'v3')
  results = drive_service.files().list(
          q="name = 'kaggle.json'", fields="files(id)").execute()
  kaggle_api_key = results.get('files', [])

  filename = "/root/.kaggle/kaggle.json"
  os.makedirs(os.path.dirname(filename), exist_ok=True)

  request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
  fh = io.FileIO(filename, 'wb')
  downloader = MediaIoBaseDownload(fh, request)
  done = False
  while done is False:
      status, done = downloader.next_chunk()
      print("Download %d%%." % int(status.progress() * 100))
  os.chmod(filename, 600)
  
  # json読み込み
  !chmod 600 /root/.kaggle/kaggle.json
  
  # kaggle読み込み
  !pip install kaggle

  # データ読み込み
  !kaggle competitions download -c ventilator-pressure-prediction

  # ファイル読み込み
  import glob
  files = glob.glob('*')
  print(files)

  if 'input' not in files:
    ! unzip train.csv.zip -d input

  if 'input' not in files:
    ! unzip test.csv.zip -d input

  if 'sample_submission' not in files:
    ! unzip sample_submission.csv.zip -d input

  # path設定
  input_path = 'input/'




elif 'kaggle_web_client' in sys.modules:
  input_path = '../input/ventilator-pressure-prediction/'


Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/__init__.py", line 44, in autodetect
    from . import file_cache
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 41, in <module>
    "file_cach

Download 100%.
 76% 57.0M/75.4M [00:01<00:00, 35.5MB/s]
100% 75.4M/75.4M [00:01<00:00, 55.8MB/s]
 86% 120M/139M [00:01<00:00, 88.0MB/s]
100% 139M/139M [00:01<00:00, 120MB/s] 
  0% 0.00/8.50M [00:00<?, ?B/s]
100% 8.50M/8.50M [00:00<00:00, 78.1MB/s]
['test.csv.zip', 'sample_submission.csv.zip', 'train.csv.zip', 'adc.json', 'sample_data']
Archive:  train.csv.zip
  inflating: input/train.csv         
Archive:  test.csv.zip
  inflating: input/test.csv          
Archive:  sample_submission.csv.zip
  inflating: input/sample_submission.csv  


In [4]:
DEBUG = False
RUN_TPU = True
MANY_FOLDS = False
SUBMIT_MEAN = False

train = reduce_mem_usage(pd.read_csv(input_path + 'train.csv'))
test = reduce_mem_usage(pd.read_csv(input_path + 'test.csv'))
submission = reduce_mem_usage(pd.read_csv(input_path + 'sample_submission.csv'))

if DEBUG:
    train = train[:80*1000]

Mem. usage decreased to 97.86 Mb (73.4% reduction)
Mem. usage decreased to 57.56 Mb (73.2% reduction)
Mem. usage decreased to 19.19 Mb (68.7% reduction)


In [5]:
def grouping_cumsum(df, stat_cumsum_cols):
    """
    cumsumを取得する

    Args:
        df(DataFrame):前処理するデータ
        stat_cumsum_cols(リスト):cumsumするリスト
        
    Returns:
        df(DataFrame)
        
    """  

    for stat_col in stat_cumsum_cols:
        
        df[stat_col + '_cumsum'] = df.groupby('breath_id')[stat_col].cumsum()

    return df

In [6]:
def grouping_describe(df, key_describe_cols, stat_describe_cols, agg_describe_kinds):
      
    """
    統計値を取得する

    Args:
        df(DataFrame):前処理するデータ
        agg_describe_kinds(リスト):統計リスト
        
    Returns:
        df(DataFrame)
        
    """  
        
    for key_col, agg_kind, stat_col in itertools.product(key_describe_cols, agg_describe_kinds, stat_describe_cols):
        
        df['breath_id__' + stat_col + '__' + agg_kind] = df.groupby([key_col])[stat_col].transform(agg_kind)
        
    return df

In [7]:
def add_features(df):
    df['area'] = df['time_step'] * df['u_in']
    df['area'] = df.groupby('breath_id')['area'].cumsum()
    
    df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
    
    
    
    print('***********************************************************************************')
    print('preprocessing...grouping_describe')
    # breath_idごとに統計値を算出する
    # df.groupby(['breath_id'])[stat_col].transform(agg_kind)
    
    key_describe_cols = [
        'breath_id'
    ]
    
    stat_describe_cols = [
        'u_in',
        'u_out'
    ]
    
    agg_describe_kinds = [
        'max',
#         'min',
        'mean', # v4
#         'std', # v4 
#         'sum' # v19
    ]

    df = grouping_describe(df, key_describe_cols, stat_describe_cols, agg_describe_kinds)
    
    print('finished')
    print('***********************************************************************************')
    
    
    
    df['u_in_lag1'] = df.groupby('breath_id')['u_in'].shift(1)
    df['u_out_lag1'] = df.groupby('breath_id')['u_out'].shift(1)
    df['u_in_lag_back1'] = df.groupby('breath_id')['u_in'].shift(-1)
    df['u_out_lag_back1'] = df.groupby('breath_id')['u_out'].shift(-1)
    df['u_in_lag2'] = df.groupby('breath_id')['u_in'].shift(2)
    df['u_out_lag2'] = df.groupby('breath_id')['u_out'].shift(2)
    df['u_in_lag_back2'] = df.groupby('breath_id')['u_in'].shift(-2)
    df['u_out_lag_back2'] = df.groupby('breath_id')['u_out'].shift(-2)
    df['u_in_lag3'] = df.groupby('breath_id')['u_in'].shift(3)
    df['u_out_lag3'] = df.groupby('breath_id')['u_out'].shift(3)
    df['u_in_lag_back3'] = df.groupby('breath_id')['u_in'].shift(-3)
    df['u_out_lag_back3'] = df.groupby('breath_id')['u_out'].shift(-3)
    df['u_in_lag4'] = df.groupby('breath_id')['u_in'].shift(4)
    df['u_out_lag4'] = df.groupby('breath_id')['u_out'].shift(4)
    df['u_in_lag_back4'] = df.groupby('breath_id')['u_in'].shift(-4)
    df['u_out_lag_back4'] = df.groupby('breath_id')['u_out'].shift(-4)
    df = df.fillna(0)
    
#     df['breath_id__u_in__max'] = df.groupby(['breath_id'])['u_in'].transform('max')
#     df['breath_id__u_out__max'] = df.groupby(['breath_id'])['u_out'].transform('max')
    
    df['u_in_diff1'] = df['u_in'] - df['u_in_lag1']
    df['u_out_diff1'] = df['u_out'] - df['u_out_lag1']
    df['u_in_diff2'] = df['u_in'] - df['u_in_lag2']
    df['u_out_diff2'] = df['u_out'] - df['u_out_lag2']
    
    df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
    df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']
    
    df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
    df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']
    
    df['u_in_diff3'] = df['u_in'] - df['u_in_lag3']
    df['u_out_diff3'] = df['u_out'] - df['u_out_lag3']
    df['u_in_diff4'] = df['u_in'] - df['u_in_lag4']
    df['u_out_diff4'] = df['u_out'] - df['u_out_lag4']
    df['cross']= df['u_in']*df['u_out']
    df['cross2']= df['time_step']*df['u_out']
    
    df['R'] = df['R'].astype(str)
    df['C'] = df['C'].astype(str)
    df['R__C'] = df["R"].astype(str) + '__' + df["C"].astype(str)
    df = pd.get_dummies(df)
    
    
    print('***********************************************************************************')
    print('preprocessing...grouping_cumsum')
    # breath_id毎に累積和を計算する
    # df.groupby('breath_id')[stat_col].cumsum()
    
    stat_cumsum_cols = [
        'u_in_diff1'
    ]
    
    df = grouping_cumsum(df, stat_cumsum_cols)
    
    print('finished')
    print('***********************************************************************************')
    
    
    
    return df

train = add_features(train)
test = add_features(test)

***********************************************************************************
preprocessing...grouping_describe
finished
***********************************************************************************
***********************************************************************************
preprocessing...grouping_cumsum
finished
***********************************************************************************
***********************************************************************************
preprocessing...grouping_describe
finished
***********************************************************************************
***********************************************************************************
preprocessing...grouping_cumsum
finished
***********************************************************************************


In [8]:
train.head()

,id,breath_id,time_step,u_in,u_out,pressure,area,u_in_cumsum,breath_id__u_in__max,breath_id__u_out__max,breath_id__u_in__mean,breath_id__u_out__mean,u_in_lag1,u_out_lag1,u_in_lag_back1,u_out_lag_back1,u_in_lag2,u_out_lag2,u_in_lag_back2,u_out_lag_back2,u_in_lag3,u_out_lag3,u_in_lag_back3,u_out_lag_back3,u_in_lag4,u_out_lag4,u_in_lag_back4,u_out_lag_back4,u_in_diff1,u_out_diff1,u_in_diff2,u_out_diff2,breath_id__u_in__diffmax,breath_id__u_in__diffmean,u_in_diff3,u_out_diff3,u_in_diff4,u_out_diff4,cross,cross2,R_20,R_5,R_50,C_10,C_20,C_50,R__C_20__10,R__C_20__20,R__C_20__50,R__C_50__10,R__C_50__20,R__C_50__50,R__C_5__10,R__C_5__20,R__C_5__50,u_in_diff1_cumsum
0,1,1,0.000000,0.083313,0,5.835938,0.000000,0.083313,28.3125,1,10.148438,0.625,0.000000,0.0,18.390625,0.0,0.000000,0.0,22.515625,0.0,0.000000,0.0,22.812500,0.0,0.000000,0.0,25.359375,0.0,0.083313,0.0,0.083313,0.0,28.234375,10.062500,0.083313,0.0,0.083313,0.0,0.0,0.0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0.083313
1,2,1,0.033661,18.390625,0,5.906250,0.619141,18.468750,28.3125,1,10.148438,0.625,0.083313,0.0,22.515625,0.0,0.000000,0.0,22.812500,0.0,0.000000,0.0,25.359375,0.0,0.000000,0.0,27.265625,0.0,18.312500,0.0,18.390625,0.0,9.921875,-8.242188,18.390625,0.0,18.390625,0.0,0.0,0.0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,18.390625
2,3,1,0.067505,22.515625,0,7.875000,2.138672,41.000000,28.3125,1,10.148438,0.625,18.390625,0.0,22.812500,0.0,0.083313,0.0,25.359375,0.0,0.000000,0.0,27.265625,0.0,0.000000,0.0,27.125000,0.0,4.125000,0.0,22.437500,0.0,5.796875,-12.367188,22.515625,0.0,22.515625,0.0,0.0,0.0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,22.515625
3,4,1,0.101562,22.812500,0,11.742188,4.453125,63.812500,28.3125,1,10.148438,0.625,22.515625,0.0,25.359375,0.0,18.390625,0.0,27.265625,0.0,0.083313,0.0,27.125000,0.0,0.000000,0.0,26.812500,0.0,0.296875,0.0,4.421875,0.0,5.500000,-12.664062,22.734375,0.0,22.812500,0.0,0.0,0.0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,22.812500
4,5,1,0.135742,25.359375,0,12.234375,7.898438,89.187500,28.3125,1,10.148438,0.625,22.812500,0.0,27.265625,0.0,22.515625,0.0,27.125000,0.0,18.390625,0.0,26.812500,0.0,0.083313,0.0,27.859375,0.0,2.546875,0.0,2.843750,0.0,2.953125,-15.210938,6.968750,0.0,25.281250,0.0,0.0,0.0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,25.359375


In [9]:
train.shape

(6036000, 56)

In [10]:
targets = train[['pressure']].to_numpy().reshape(-1, 80)
train.drop(['pressure', 'id', 'breath_id'], axis=1, inplace=True)
test = test.drop(['id', 'breath_id'], axis=1)

In [11]:
RS = RobustScaler()
train = RS.fit_transform(train)
test = RS.transform(test)

In [12]:
train = train.reshape(-1, 80, train.shape[-1])
test = test.reshape(-1, 80, train.shape[-1])

In [13]:
if RUN_TPU == True:

  EPOCH = 300
  BATCH_SIZE = 1024

  if MANY_FOLDS == True:
    NUM_FOLDS = 10
  else:
    NUM_FOLDS = 5
  print('NUM_FOLDS:', NUM_FOLDS)

  # detect and init the TPU
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

  # instantiate a distribution strategy
  tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)


  with tpu_strategy.scope():
      kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=2021)
      test_preds = []
      for fold, (train_idx, test_idx) in enumerate(kf.split(train, targets)):
          print('-'*15, '>', f'Fold {fold+1}', '<', '-'*15)
          X_train, X_valid = train[train_idx], train[test_idx]
          y_train, y_valid = targets[train_idx], targets[test_idx]
          model = keras.models.Sequential([
              keras.layers.Input(shape=train.shape[-2:]),
              keras.layers.Bidirectional(keras.layers.LSTM(1024, return_sequences=True)),
              keras.layers.Bidirectional(keras.layers.LSTM(512, return_sequences=True)),
              keras.layers.Bidirectional(keras.layers.LSTM(256, return_sequences=True)),
              keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True)),
              keras.layers.Dense(128, activation='selu'),
              keras.layers.Dense(1),
          ])
          model.compile(optimizer="adam", loss="mae")

          lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=10, verbose=1)
          es = EarlyStopping(monitor="val_loss", patience=30, verbose=1, mode="min", restore_best_weights=True)
      
          checkpoint_filepath = f"folds{fold}.hdf5"
          sv = keras.callbacks.ModelCheckpoint(
              checkpoint_filepath, monitor='val_loss', verbose=1, save_best_only=True,
              save_weights_only=False, mode='auto', save_freq='epoch',
              options=None
          )

          model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=EPOCH, batch_size=BATCH_SIZE, callbacks=[lr, es, sv])
          test_preds.append(model.predict(test).squeeze().reshape(-1, 1).squeeze())

NUM_FOLDS: 5
INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.125.214.34:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.125.214.34:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


--------------- > Fold 1 < ---------------
Epoch 1/300
59/59 [==============================] - 100s 1s/step - loss: 2.6742 - val_loss: 1.3324

Epoch 00001: val_loss improved from inf to 1.33243, saving model to folds0.hdf5
Epoch 2/300
59/59 [==============================] - 39s 655ms/step - loss: 1.0037 - val_loss: 0.7187

Epoch 00002: val_loss improved from 1.33243 to 0.71867, saving model to folds0.hdf5
Epoch 3/300
59/59 [==============================] - 38s 652ms/step - loss: 0.6903 - val_loss: 0.7129

Epoch 00003: val_loss improved from 0.71867 to 0.71288, saving model to folds0.hdf5
Epoch 4/300
59/59 [==============================] - 39s 653ms/step - loss: 0.5904 - val_loss: 0.4987

Epoch 00004: val_loss improved from 0.71288 to 0.49865, saving model to folds0.hdf5
Epoch 5/300
59/59 [==============================] - 39s 653ms/step - loss: 0.5014 - val_loss: 0.4939

Epoch 00005: val_loss improved from 0.49865 to 0.49388, saving model to folds0.hdf5
Epoch 6/300
59/59 [=========

In [14]:
if RUN_TPU == False:

  EPOCH = 300
  BATCH_SIZE = 1024

  if MANY_FOLDS == True:
    NUM_FOLDS = 10
  else:
    NUM_FOLDS = 5
  print('NUM_FOLDS:', NUM_FOLDS)

  gpu_strategy = tf.distribute.get_strategy()

  with gpu_strategy.scope():
      kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=2021)
      test_preds = []
      for fold, (train_idx, test_idx) in enumerate(kf.split(train, targets)):
          print('-'*15, '>', f'Fold {fold+1}', '<', '-'*15)
          X_train, X_valid = train[train_idx], train[test_idx]
          y_train, y_valid = targets[train_idx], targets[test_idx]
          model = keras.models.Sequential([
              keras.layers.Input(shape=train.shape[-2:]),
              keras.layers.Bidirectional(keras.layers.LSTM(1024, return_sequences=True)),
              keras.layers.Bidirectional(keras.layers.LSTM(512, return_sequences=True)),
              keras.layers.Bidirectional(keras.layers.LSTM(256, return_sequences=True)),
              keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True)),
              keras.layers.Dense(128, activation='selu'),
              keras.layers.Dense(1),
          ])
          model.compile(optimizer="adam", loss="mae")

          lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=10, verbose=1)
          es = EarlyStopping(monitor="val_loss", patience=30, verbose=1, mode="min", restore_best_weights=True)
      
          checkpoint_filepath = f"folds{fold}.hdf5"
          sv = keras.callbacks.ModelCheckpoint(
              checkpoint_filepath, monitor='val_loss', verbose=1, save_best_only=True,
              save_weights_only=False, mode='auto', save_freq='epoch',
              options=None
          )

          model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=EPOCH, batch_size=BATCH_SIZE, callbacks=[lr, es, sv])
          test_preds.append(model.predict(test).squeeze().reshape(-1, 1).squeeze())

In [15]:
!ls ./

adc.json     folds2.hdf5  input			     test.csv.zip
folds0.hdf5  folds3.hdf5  sample_data		     train.csv.zip
folds1.hdf5  folds4.hdf5  sample_submission.csv.zip


In [16]:
submission["pressure"] = sum(test_preds)/NUM_FOLDS
submission.to_csv('submission_mean.csv', index=False)

if ('google.colab' in sys.modules) & (SUBMIT_MEAN == True):
  !kaggle competitions submit ventilator-pressure-prediction -f submission_mean.csv -m "Yeah! I submit my file through the Google Colab!"

In [17]:
submission["pressure"] = np.median(np.vstack(test_preds),axis=0)
submission.to_csv('submission_median.csv', index=False)

if 'google.colab' in sys.modules:
  !kaggle competitions submit ventilator-pressure-prediction -f submission_median.csv -m "Yeah! I submit my file through the Google Colab!"

100% 66.3M/66.3M [00:03<00:00, 22.2MB/s]
Successfully submitted to Google Brain - Ventilator Pressure Prediction